# Abstract

Instance Segmentation을 이해함에 있어서 그 기저가 되는 Object Detection과 Segmentation을 이해하는 것은 필수이다. 이번 Report는 Object Detection의 개념과 작동원리에 대해서 명확하게 이해하는 것이 목적이다.

# Object Localization

<img src="https://i.imgur.com/bGpdSjp.png" width="100%">

Object Detection을 이해하기 위해서는 그것의 기저인 Image Classification과 Localization을 이해해야한다. 맨 왼쪽 그림은 Image Classification으로 단순히 Image 내의 물체가 어떤 물체인지 분류해준다. 나아가 가운데 그림은 Image 내의 물체를 분류해주고 그 물체의 Bounding Box를 잡아준다. 이 때 지금까지 언급한 두 문제는 Image내의 하나의 물체만을 대상으로 한다. 만약 그 대상이 많아진다면 물체 각각에 대하여 분류해주고 Bounding Box를 잡아주어야 할 것이다. 이것이 바로 Object Detection이다.

<img src="https://i.imgur.com/79QEXs2.png" width="100%">

위에서 언급한 문제 중 'Classification with Localization' 문제를 좀 더 살펴보자. 모델은 위와 같이 구성된다. 마지막 출력층을 보면 기존 Classification 문제를 해결하기 위한 softmax 분류 layer 뿐만 아니라 물체의 Bounding Box를 출력하는 층이 포함된다. 해당 층은 총 4개의 결과를 출력하는데, Bounding Box의 중심좌표에 해당하는 bx, by, Bounding Box의 높이와 너비에 해당하는 bh, bw가 그것이다. 특히 분류를 함에 있어서 Bounding Box안에 물체가 없을 경우가 있으므로 background에 해당하는 클래스를 하나 추가해준다.

<img src="https://i.imgur.com/CTwRVx9.png" width="100%">

target label을 만드는 방법은 위와 같다. y 벡터는 물체의 존재 여부(물체가 있는지 나타내는 확률)를 표현하는 pc, Bounding Box의 정보를 표현하는 bx, by, bh, bw, 그리고 클래스 별 존재확률인 c1, c2, c3, ... 으로 구성된다. 특히 pc의 경우 물체가 있다면 1, 없다면 0의 값을 갖는데, 만약 물체가 없다면 나머지 원소들의 값이 무의믜하므로 신경쓰지 않는다. 따라서 Loss Function의 구성을 생각해보자면 다음과 같다. 먼저 물체가 존재하는 경우(y=1)는 y_hat과 y벡터의 L2 Loss를 통해 그 차이를 계산할 수 있다. 물체가 존재하지 않는 경우(y=0)는 pc의 값만 유의미하기 때문에 y_hat과 y 벡터의 pc에 해당하는 원소만 제곱 오차를 사용하여 그 차이를 계산한다. 물론 이는 가장 간편한 방법이고, 설계에 따라서 pc부분을 로지스틱 회귀 손실을 사용할 수도 있고, c1, c2, c3, ...부분을 로그 우도 손실을 사용할 수  도 있다.

# Landmark Detection

<img src="https://i.imgur.com/buozZbl.png" width="100%">

그 다음 살펴볼 내용은 Landmark Detection(특징점 검출)이다. 흔히 Face Recognition 문제나 Pose Estimation 문제가 이에 해당한다. 이는 Object Localization과는 달리 얼굴 인식에 중요한 특징점 또는 자세 추정에 중요한 특징점 등 문제를 해결하기에 중요하다고 생각하는 특징점들의 좌표를 예측하는 문제이다. 따라서 y 벡터는 물체의 존재 여부(물체가 있는지 나타내는 확률)를 표현하는 pc, 특징점들의 좌표를 표현하는 l1x, l1y, l2x, l2y, ...로 구성되어있다. 예를 들어 위의 가운데 이미지는 Face Recognition 문제에 해당하는데, 얼굴의 존재여부를 표현하는 원소, 그리고 얼굴인식에 중요하다고 생각하는 64개 특징점의 좌표를 표현하는 원소로 이루어져 총 129개의 원소를 가지는 y 벡터가 필요할 것이다. 특히 중요한 점은 특징점은 다른 이미지에 대해서도 항상 동일해야 한다는 점이다. 예를 들어 특징점 1은 항상 왼쪽 눈의 눈꼬리가 되어야 하고, 특징점 2는 항상 오른쪽 눈의 눈꼬리가 되어야 한다. 이렇듯 레이블 순서는 다른 이미지에 대해서도 항상 동일하게 구성되어 있어야 한다.

# Object Detection

<img src="https://i.imgur.com/3mVAGcC.png" width="100%">

이제 Object Detection에 대해서 살펴보자. 이는 물체의 위치를 탐지하여 BBox를 그리고 물체를 분류하는 문제라고 앞에서 언급했다. 기본적인 작동방식은 위의 그림과 같다. 먼저 전체 Image에서 자동차가 있을 만한 곳의 BBox를 모두 잡는다. 물론 자동차를 잡지 못한 BBox 또한 존재할 것이다. 이 모든 BBox를 ConvNet에 집어넣어 자동차인지를 분류하면 된다. 요약하자면, 물체가 있을만한 곳의 BBox를 모두 잡아내고, 그것들을 모두 ConvNet에 집어넣어 물체의 존재여부를 분류한다. 마찬가지로 ConvNet을 훈련시키기 위해서는 배경을 제외한 자동차만 존재하는 BBox와 그 레이블 데이터를 준비해야 할 것이다.

<img src="https://i.imgur.com/D3aQAek.png" width="100%">

그렇다면 자동차가 있을만한 곳의 BBox는 어떻게 잡는걸까? 이 방법 중의 하나로 'Sliding windows detection'이 있다. 미리 정해둔 크기의 window를 Image상에서 일정간격마다 적용시켜 수많은 BBox를 만드는 방법이다. 이후 추출한 BBox들을 ConvNet에 집어넣으면 될 것이다. 간단해보이는 방법이지만 단점이 존재한다. 이 방법은 이미지의 수많은 영역을 모두 잘라내야 하고 합성곱 신경망을 통해 이것들을 각각 계산해야 하기 때문에 시간과 비용의 문제가 발생한다. 매우 정밀한 입도 또는 이동간격을 사용한다면 모든 작은 영역을 합성곱 신경망에 통과시켜야 하기 때문에 매우 높은 계산 비용이 필요하다. 반대로 매우 큰 슬라이드 간격을 사용한다면 합성곱 신경망을 통과시켜야 하는 window의 수는 줄어들지만 물체를 제대로 탐지하지 못해 성능이 저하되는 문제가 발생한다.

# Convolutional Implementation Sliding Windows

앞서 제시한 Sliding Windows 방법은  Window 각각을 ConvNet에 입력해야 하기 때문에 시간이 오래 걸린다는 단점을 확인했다. 하지만 이는 Convolutional Implementation을 통해 그 문제를 해결할 수 있는데, 살펴보도록 하자.

<img src="https://i.imgur.com/Ubwnwb9.png" width="100%">

먼저 FC가 어떻게 Convolutional Layer로 바뀔 수 있는지 살펴보자. 위의 그림의 첫 번째는 FC Layers를 의미하고, 두 번째는 Convolutional Layers를 의미한다. FC Layers에서는 5x5x16의 feature map을 단순히 FC를 이용하여 400개의 출력을 만들어낸다. 반면에 이를 400 filter의 5x5 conv를 이용한다면 1x1x400의 출력을 만들어내어 마치 FC와 같은 기능을 하는 역할을 한다. 핵심은 1x1 conv라고 할 수 있겠다.

<img src="https://i.imgur.com/BEz4j2G.png" width="100%">

자 그럼 이 방법을 이용해서 무슨 이점을 얻을 수 있는지 살펴보자. 위의 그림의 첫 번째 내용은 14x14x3 Image에 대하여 적용한 내용으로써 이전 슬라이드와 같다. 두 번째, 세 번째의 Image에 14x14 사이즈의 Sliding Window를 적용한다고 가정해보자.<br>
두 번째 내용 기준으로, 만약 우리가  Convolutional Implementation을 사용하지 않는다면 총 4개의 Window를 ConvNet에 적용시킬것이다. 예시이기 때문에 작은 사이즈 Image에 적용했지만 만약 더 큰 사이즈의 Image라면 더 많은 수의 Window를 ConvNet에 적용시켜야 하기 때문에 시간이 오래 걸린다. 또한 각 Window는 겹치는 부분이 많아서 ConvNet이 많은 반복 계산을 수행한다는 비효율이 발생한다.<br>
Convolutional Implementation을 사용한다면 어떻게 될까? 16x16x3 Image에 첫 번째 내용과 같은 구조의 모델을 적용시킨다면 총 2x2x4의 출력을 얻을 수 있는데, 이는 곧 4개의 Window를 ConvNet에 입력시켜야하는 과정을 한 번에 수행할 수 있다는 것을 보여준다. 출력값의 (1,1)은 첫 번째 Window를 적용한 값을 나타내고, 출력값의 (1,2)는 두 번째 Window를 적용한 값을 나타낸다고 볼 수 있겠다. 즉, 연산과정에서 겹치는 반복계산을 공유하는 이점을 가질 수 있다. 그리고 각 위치에 대해서 4개의 값을 갖게 되는데, 이는 마치 Softmax함수를 수행한 것과 같은 효과를 얻을 수 있다. 즉, 각각의 Window가 어느 클래스를 나타내는지 분류하는 문제와 같다.<br>
마찬가지로 세 번째 내용을 살펴보면 Sliding Window를 통해 64개의 filter map을 ConvNet에 적용시키는 과정을 단 한 번에 수행하는 것을 보여주고 있다고 할 수 있겠다.

<img src="https://i.imgur.com/4E7ANOT.png" width="100%">

정리하자면, Convolutional Implementation을 이용하여 각각의 Window를 ConvNet에 집어넣는 과정을 단 한번에 수행할 수 있는 모델을 만들 수 있다. 하지만 이 알고리즘은 여전히 문제를 가지고 있는데, BBox의 위치가 정확하지 않을 수 있다는 것이 그것이다.

# Reference

deeplearning.ai
* https://www.deeplearning.ai/
* https://www.youtube.com/channel/UCcIXc5mJsHVYTZR1maL5l9w